<a href="https://colab.research.google.com/github/Roshni19sharma99/LLM/blob/main/Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [49]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle
import pandas as pd
import numpy as np

In [50]:
#### load the trained model, scalar pickle onehot
model = load_model('model.h5')

##load the encoder and scalar
with open('onehot_encoder_geo.pkl','rb') as file:
  label_encoder_geo = pickle.load(file)

with open('label_encoder_gender.pkl','rb') as file:
  label_encoder_gender = pickle.load(file)

with open('scalar.pkl','rb') as file:
  scaler = pickle.load(file)


In [51]:
input_data = {
  'CreditScore' : 600,
  'Geography' : 'France',
  'Gender' : 'Male',
  'Age' : 40,
  'Tenure' : 3,
  'Balance' : 60000,
  'NumOfProducts' : 2,
  'HasCrCard' : 1,
  'IsActiveMember' : 1,
  'EstimatedSalary' : 50000
}

In [52]:
#one-hot encode 'Geography'
geo_encoded = label_encoder_geo.transform([[input_data['Geography']]]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded,columns=label_encoder_geo.get_feature_names_out(['Geography']))
geo_encoded_df

/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [53]:
input_df = pd.DataFrame([input_data])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,40,3,60000,2,1,1,50000


In [54]:
input_df['Gender'] = label_encoder_gender.transform(input_df['Gender'])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,1,40,3,60000,2,1,1,50000


In [55]:
input_df=pd.concat([input_df.drop("Geography",axis=1),geo_encoded_df],axis=1)
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [56]:
input_scaled = scaler.transform(input_df)
input_scaled

array([[-0.52046273,  0.91509065,  0.10629772, -0.70174202, -0.26396987,
         0.80225696,  0.64376017,  0.97725852, -0.87097985,  1.00150113,
        -0.58312392, -0.57273139]])

In [57]:
prediction = model.predict(input_scaled)
prediction

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 142ms/step


array([[0.04482377]], dtype=float32)

In [60]:
prediction_proba = prediction[0][0]

In [61]:
prediction_proba

np.float32(0.044823766)

In [62]:
if prediction_proba<0.5:
  print('The customer is likely to churn')
else:
  print('The customer is not likely to churn')

The customer is not likely to churn
